In [1]:
import numpy as np
from cities.utils.data_grabber import find_repo_root, MSADataGrabberCSV, DataGrabberCSV
import pandas as pd
import numpy as np
import requests
from us import states

root = find_repo_root()

data = DataGrabberCSV()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]

dataMA = MSADataGrabberCSV()
dataMA.get_features_wide(["gdp_ma"])
gdp_ma = dataMA.wide["gdp_ma"]


# note that there is also a library for accessing the Census data:
# https://github.com/datamade/census

In [2]:
# age variables, firstly let's focus just on the total population
# ,S0101_C01_002E,S0101_C01_003E,S0101_C01_004E,S0101_C01_005E,S0101_C01_006E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_015E,S0101_C01_016E,S0101_C01_017E,S0101_C01_018E,S0101_C01_019E

variables =  "NAME,S0101_C01_001E"
county_fips = "*" # all counties
tract = "*" # all tracts
api_key = '077d857d6c12d5b9b3aeafa07d2c1916ba12a86c' # private api key required to access the data https://api.census.gov/data/key_signup.html
year = 2021


interval = list(range(2010, 2023))  # Time series for ACS 5-year estimates

dfs = []


for year in interval:
    for x in range(0, len(states.STATES)): # in this call it's not possible to use the '*' wildcard to access all states, so we need to iterate over all states
        fips = states.STATES[x].fips

        url = f'https://api.census.gov/data/{year}/acs/acs5/subject?get={variables}&for=tract:{tract}&in=state:{fips}&in=county:{county_fips}&key={api_key}'

        response = requests.get(url)

        assert response.status_code == 200, 'The data retrieval went wrong'  # 200 means success

        print(f'{fips} fips for year {year} done')

        data = response.json()

        df = pd.DataFrame(data[1:], columns=data[0])
        df['Year'] = year  # Add the year column

        dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)


01 fips for year 2010 done
02 fips for year 2010 done
04 fips for year 2010 done
05 fips for year 2010 done
06 fips for year 2010 done
08 fips for year 2010 done
09 fips for year 2010 done
10 fips for year 2010 done
12 fips for year 2010 done
13 fips for year 2010 done
15 fips for year 2010 done
16 fips for year 2010 done
17 fips for year 2010 done
18 fips for year 2010 done
19 fips for year 2010 done
20 fips for year 2010 done
21 fips for year 2010 done
22 fips for year 2010 done
23 fips for year 2010 done
24 fips for year 2010 done
25 fips for year 2010 done
26 fips for year 2010 done
27 fips for year 2010 done
28 fips for year 2010 done
29 fips for year 2010 done
30 fips for year 2010 done
31 fips for year 2010 done
32 fips for year 2010 done
33 fips for year 2010 done
34 fips for year 2010 done
35 fips for year 2010 done
36 fips for year 2010 done
37 fips for year 2010 done
38 fips for year 2010 done
39 fips for year 2010 done
40 fips for year 2010 done
41 fips for year 2010 done
4

In [4]:
combined_df.to_csv('census_tracts_2010_2022.csv', index=False)

In [63]:
pop = combined_df.copy()
combined_df.shape[0]

981397

In [64]:
column_mapping = {
    'S0101_C01_001E': 'Value'
}

pop.rename(columns=column_mapping, inplace=True)
pop.head()

,NAME,Value,state,county,tract,Year
0,"Census Tract 110.01, Etowah County, Alabama",4435,01,055,011001,2010
1,"Census Tract 110.02, Etowah County, Alabama",4718,01,055,011002,2010
2,"Census Tract 111, Etowah County, Alabama",4796,01,055,011100,2010
3,"Census Tract 112, Etowah County, Alabama",2691,01,055,011200,2010
4,"Census Tract 200, Fayette County, Alabama",1572,01,057,020000,2010


In [65]:
state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

pop['GeoFIPS'] = pop.apply(lambda row: f"{row['state']}{row['county']}{row['tract']}", axis=1).astype(np.int64)

pop.drop(['state', 'county', 'tract'], axis=1, inplace=True)

#pop['GeoName'] = pop['NAME'].apply(lambda x: f"{x.split(',')[1].strip().replace(' County', '')}, {state_abbreviations[x.split(',')[2].strip()]} (CT)")


def parse_geo_name(name):
    if ';' in name:
        parts = name.split(';')
    else:
        parts = name.split(',')

    if len(parts) >= 3:
        county = parts[1].strip().replace(' County', '')
        state_full = parts[2].strip()
        state_abbr = state_abbreviations.get(state_full, state_full)  
        return f"{county}, {state_abbr} (CT)"
    return "Unknown"


pop['GeoName'] = pop['NAME'].apply(parse_geo_name).astype(str)

assert pop[pop['GeoName'] == 'Unknown'].shape[0] == 0, 'There are Unknown GeoNames'

pop = pop.drop(['NAME'], axis=1)

pop.head()

,Value,Year,GeoFIPS,GeoName
0,4435,2010,1055011001,"Etowah, AL (CT)"
1,4718,2010,1055011002,"Etowah, AL (CT)"
2,4796,2010,1055011100,"Etowah, AL (CT)"
3,2691,2010,1055011200,"Etowah, AL (CT)"
4,1572,2010,1057020000,"Fayette, AL (CT)"


In [66]:
len(pop['GeoName'].unique())

3158

In [67]:
pop.sort_values(by=['Year', 'GeoFIPS', 'GeoName'], inplace=True)
pop = pop[['GeoFIPS', 'GeoName', 'Year', 'Value']].reset_index(drop=True)

pop.head()

,GeoFIPS,GeoName,Year,Value
0,1001020100,"Autauga, AL (CT)",2010,1809
1,1001020200,"Autauga, AL (CT)",2010,2020
2,1001020300,"Autauga, AL (CT)",2010,3543
3,1001020400,"Autauga, AL (CT)",2010,4840
4,1001020500,"Autauga, AL (CT)",2010,9938


The number of Census Tracts recorded in each year

In [68]:
unique_geofips_per_year = pop.groupby('Year')['GeoFIPS'].nunique()
unique_geofips_per_year

Year
2010    72878
2011    72878
2012    72877
2013    72877
2014    72877
2015    72877
2016    72877
2017    72877
2018    72877
2019    72877
2020    84208
2021    84208
2022    84209
Name: GeoFIPS, dtype: int64

The number of Census Tracts that are present across the years

In [72]:
geofips_counts = pop.groupby('GeoFIPS')['Year'].nunique()

num_years = pop['Year'].nunique()

geofips_in_each_year = geofips_counts[geofips_counts == num_years].index.tolist()

geofips_in_each_year_df = pop[pop['GeoFIPS'].isin(geofips_in_each_year)][['GeoFIPS', 'GeoName']].drop_duplicates(subset=['GeoFIPS'])

print("GeoFIPS values that are present in each year:")
print(geofips_in_each_year_df.shape[0])

GeoFIPS values that are present in each year:
60693


In [69]:
pop_wide = pop.pivot(index=['GeoFIPS', 'GeoName'], columns='Year', values='Value')
pop_wide = pop_wide.reset_index()
pop_wide.columns.name = None
pop_wide.head()

,GeoFIPS,GeoName,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,1001020100,"Autauga, AL (CT)",1809,1768,1812,1808,1900,1948,2010,1845,1923,1993,1941,1791,1865
1,1001020200,"Autauga, AL (CT)",2020,2167,2218,2355,2342,2156,2196,2172,2028,1959,1757,2010,1861
2,1001020300,"Autauga, AL (CT)",3543,3337,3155,3057,3297,2968,3136,3385,3476,3507,3694,3577,3492
3,1001020400,"Autauga, AL (CT)",4840,4379,4337,4403,4272,4423,4563,4267,3831,3878,3539,3802,3987
4,1001020500,"Autauga, AL (CT)",9938,10197,10498,10851,10881,10763,10529,9965,9883,10596,NaN,NaN,NaN


Creating the Source of Truth for Census Tracts

In [55]:
unique_geofips_df = pop.drop_duplicates(subset=['GeoFIPS'])
unique_geofips_df = unique_geofips_df[['GeoFIPS', 'GeoName']].reset_index(drop=True)

unique_geofips_df

,GeoFIPS,GeoName
0,1001020100,"Autauga, AL (CT)"
1,1001020200,"Autauga, AL (CT)"
2,1001020300,"Autauga, AL (CT)"
3,1001020400,"Autauga, AL (CT)"
4,1001020500,"Autauga, AL (CT)"
...,...,...
96506,9190253400,"Western Connecticut Planning Region, CT (CT)"
96507,9190253500,"Western Connecticut Planning Region, CT (CT)"
96508,9190257100,"Western Connecticut Planning Region, CT (CT)"
96509,9190425700,"Western Connecticut Planning Region, CT (CT)"
